Stopwords source: https://github.etsycorp.com/Engineering/BigData/blob/master/operators/src/main/resources/nlp/stopwords/en-full.txt

In [1]:
!gcloud config set project etsy-bigquery-adhoc-prod


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/project].


In [2]:
%load_ext google.cloud.bigquery

generate table: sr_training_data_queries.sql

In [3]:
%%bigquery df
select distinct data_date, requestUUID, client_query
from `etsy-sr-etl-prod.yzhang.attributed_instance_queries_1229_0104`

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 407397/407397 [00:03<00:00, 135282.36rows/s]


In [4]:
df.head()

,data_date,requestUUID,client_query
0,2023-12-29,00016f24-7795-47d2-9b41-f75d02d1aabe,disney princess png
1,2023-12-29,0001e116-736a-4dcf-ae02-2eb20c6c9a1f,disney princess thank you tag
2,2023-12-29,00024aa4-1925-41ce-8ef6-703ece70f6d3,niv bible
3,2023-12-29,000252b9-d077-46a5-b457-25a37fab6a37,gift
4,2023-12-29,0002b0ed-b8db-4359-bfcb-7d0b1357b3d7,joseph 14birthday figurines doll


In [5]:
df.client_query.value_counts()

digital planner                                627
stanley name plate                             385
monopoly go stickers                           331
2024 digital planner                           277
2024 calendar                                  234
                                              ... 
metal conversation hearts                        1
magritte lovers                                  1
katescrochetpatterns                             1
small fridge magnet calendars                    1
etsy printable sign pickleball and prosecco      1
Name: client_query, Length: 294062, dtype: int64

# Stopwords

In [6]:
stopwords_full = set()
tmp = []
with open("./stopwords_full.txt", "r") as f:
    for line in f:
        stopwords_full.add(line.strip())
        tmp.append(line.strip())

In [7]:
len(stopwords_full)

612

In [8]:
stopwords_sub = set()
with open("./stopwords.txt", "r") as f:
    for line in f:
        stopwords_sub.add(line.strip())

In [9]:
len(stopwords_sub)

34

In [10]:
def check_stopword_overlap(qstring, stopwd_set):
    query_split = qstring.split(" ")
    for word in query_split:
        if word in stopwd_set:
            return word
    return None

In [11]:
df["stopword_full_overlap"] = df.client_query.apply(check_stopword_overlap, stopwd_set=stopwords_full)
df["stopword_sub_overlap"] = df.client_query.apply(check_stopword_overlap, stopwd_set=stopwords_sub)

In [12]:
df.head()

,data_date,requestUUID,client_query,stopword_full_overlap,stopword_sub_overlap
0,2023-12-29,00016f24-7795-47d2-9b41-f75d02d1aabe,disney princess png,None,None
1,2023-12-29,0001e116-736a-4dcf-ae02-2eb20c6c9a1f,disney princess thank you tag,thank,None
2,2023-12-29,00024aa4-1925-41ce-8ef6-703ece70f6d3,niv bible,None,None
3,2023-12-29,000252b9-d077-46a5-b457-25a37fab6a37,gift,None,None
4,2023-12-29,0002b0ed-b8db-4359-bfcb-7d0b1357b3d7,joseph 14birthday figurines doll,None,None


In [13]:
df[~df.stopword_full_overlap.isna()].shape

(89917, 5)

In [14]:
df.data_date.value_counts()

2024-01-03    62348
2024-01-02    62283
2024-01-04    61204
2023-12-29    59766
2023-12-30    58021
2023-12-31    54251
2024-01-01    49524
Name: data_date, dtype: int64

In [15]:
df[~df.stopword_full_overlap.isna()].data_date.value_counts()

2024-01-03    13523
2024-01-04    13278
2023-12-29    13231
2024-01-02    13133
2023-12-30    12984
2023-12-31    12908
2024-01-01    10860
Name: data_date, dtype: int64